In [2]:
import sys, os
import time
import numpy as np
path_current = '/home/huzuntao/PycharmProjects/MPP_Powersystem/'
path_ = os.getcwd()
if path_current not in sys.path:
    sys.path.insert(1, '/home/huzuntao/PycharmProjects/MPP_Powersystem/')
elif path_ not in sys.path:
    sys.path.insert(1, path_)

from PPOPT_main.PPOPT_main.src.ppopt.mpQCQP_program import MPQCQP_Program
from PPOPT_main.PPOPT_main.src.ppopt.mp_solvers.solver_utils import generate_children_sets, CombinationTester
from PPOPT_main.PPOPT_main.src.ppopt.solution import Solution
from PPOPT_main.PPOPT_main.src.ppopt.utils.mpqp_utils import gen_cr_from_active_set

#### !pip install numpy
#### 记得用conda安装

In [3]:
import sys
print(sys.version)

3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]


In [4]:
import pandapower as pp
net = pp.create_empty_network()

# create buses
bus1 = pp.create_bus(net, vn_kv=110.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
# bus4 = pp.create_bus(net, vn_kv=110.)
# bus5 = pp.create_bus(net, vn_kv=110.)

# create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=90., std_type='149-AL1/24-ST1A 110.0')
# pp.create_line(net, bus3, bus4, length_km=50., std_type="149-AL1/24-ST1A 110.0")
# pp.create_line(net, bus4, bus2, length_km=40., std_type="149-AL1/24-ST1A 110.0")
pp.create_line(net, bus1, bus2, length_km=70., std_type='149-AL1/24-ST1A 110.0')

# create loads
pp.create_load(net, bus2, p_mw=60., controllable=False)
# pp.create_load(net, bus3, p_mw=70., controllable=False)
# pp.create_load(net, bus4, p_mw=25., controllable=False)

# create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=0, max_p_mw=1000, vm_pu=1.05)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80, vm_pu=1.00, controllable=True)
# g1 = pp.create_gen(net, bus4, p_mw=50, min_p_mw=0, max_p_mw=50, vm_pu=1.00, controllable=True)


costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=20)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

net.bus["min_vm_pu"] = 0.96
net.bus["max_vm_pu"] = 1.04
net.line["max_loading_percent"] = 100
# net.sn_mva = 100
# net = pp.networks.case14()
# net.gen['vm_pu'] = 1.0
# net.bus["min_vm_pu"] = 0.95
# net.bus["max_vm_pu"] = 1.05
# net.poly_cost[:,3] = 0
# net.poly_cost['cp2_eur_per_mw2'] = 0
net.poly_cost

/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/pandapower/auxiliary.py:268: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item, dtype in list(dtypes.iteritems()):
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,10.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,10.0,0.0,0.0,0.0,0.0


In [5]:
my_QCQP = MPQCQP_Program(net=net)

The constraint: [<6x10 sparse matrix of type '<class 'numpy.float64'>'
 	with 32 stored elements in List of Lists format>
 <4x10 sparse matrix of type '<class 'numpy.float64'>'
 	with 16 stored elements in List of Lists format>    ]
flow_max: [8018.67+0.j 8018.67+0.j]


/home/huzuntao/PycharmProjects/MPP_Powersystem/pypower_/makeYbus.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  tap[i] = branch[i, TAP]  ## assign non-zero tap ratios
/home/huzuntao/.conda/envs/nvenv/lib/python3.9/site-packages/scipy/sparse/_coo.py:159: ComplexWarning: Casting complex values to real discards the imaginary part
  self.col = np.array(col, copy=copy, dtype=idx_dtype)


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 10, expected 4.

In [29]:
# convert sparse matrix to dense matrix in numpy
dense_matrix = my_QCQP.raw['dg']
dense_matrix = dense_matrix[0].toarray()
dense_matrix
# 判断密集矩阵是否为稠密矩阵
# if dense_matrix.ndim == 2 and dense_matrix.dtype in [np.float32, np.float64]:
#     print("dense_matrix is a dense matrix")
# else:
#     print("dense_matrix is not a dense matrix")

array([[ 358.00683452, -358.00683452,    0.        ,  171.14654799,
        -179.51146633,    0.        ,   -1.        ,    0.        ,
           0.        ,    0.        ],
       [-365.97526   ,  611.05093962, -245.07567962, -155.10792884,
         230.14659243, -180.31730811,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        , -300.4131747 ,  300.4131747 ,    0.        ,
         -70.50394738,  195.82728281,    0.        ,   -1.        ,
           0.        ,    0.        ],
       [-179.70381213,  179.70381213,    0.        ,  379.99918896,
        -357.62364226,    0.        ,    0.        ,    0.        ,
          -1.        ,    0.        ],
       [ 162.86332528, -350.39319354,  187.52986826, -348.54786667,
         610.39690185, -235.64985803,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,   70.57949207,  -70.57949207,    0.        ,
        -300.09162775,  265.31538583,    0.        ,    0

In [9]:
from numpy import flatnonzero as find
from scipy.sparse import vstack, hstack, eye, diags, bmat, csc_matrix, csr_matrix as sparse

mu_ = my_QCQP.raw['lmbda']['mu']
idx_act = find(mu_ > 0)
mu_[idx_act]

array([320.51104468,   7.40918473])

In [10]:
import numpy as np
from scipy import linalg
rng = np.random.default_rng()
a = rng.standard_normal((9000, 6000))
B = linalg.pinv(a)
np.allclose(a, a @ B @ a)

True

In [18]:
from numpy import r_, ones
xmin = np.array([1,2,3])
l = None
if l is None or len(l) == 0: l = -1 * ones(0)
ll = r_[xmin, l]
ll


dtype('float64')

In [18]:
import numpy as np
from scipy.sparse import csr_matrix

# 创建一个稀疏矩阵
sparse_matrix = csr_matrix(([1, 2, 3], ([0, 1, 2], [1, 2, 3])), shape=(3, 4))
print("稀疏矩阵：\n", sparse_matrix)

# 将稀疏矩阵转换为密集矩阵
dense_matrix = sparse_matrix.toarray()
print("密集矩阵：\n", dense_matrix)


稀疏矩阵：
   (0, 1)	1
  (1, 2)	2
  (2, 3)	3
密集矩阵：
 [[0 1 0 0]
 [0 0 2 0]
 [0 0 0 3]]


In [5]:
n_constrs = my_QCQP.num_constraints()
n_eq = my_QCQP.num_equality_constraints()
n_theta = my_QCQP.num_t() # correct
n_variables = my_QCQP.num_x()
print(f'There are {n_constrs} constraints, where {n_eq} are equalities;'
      f'\nThere are {n_variables} variables and {n_theta} parameters.')

There are 40 constraints, where 28 are equalities;
There are 30 variables and 3 parameters.


In [6]:
# https://www.sciencedirect.com/science/article/pii/S0005109811003190
# solve the MPQCQP, based on mpqp_combinational in PPOPT_main

program = my_QCQP
murder_list = CombinationTester()
to_check = list()

solution = Solution(program, [])

max_depth = max(program.num_x(), program.num_t()) - len(program.equality_indices)
# breath first to optimize the elimination

root_node = generate_children_sets(program.equality_indices, program.num_constraints(), murder_list)

to_check.extend(root_node)

In [7]:
# 现在是第一层，一共20个不等式约束， 每个不等式约束为一个根节点
len(root_node)

12

In [8]:
max_depth   # 这个最大深度的公式和excel

2

In [9]:
from typing import List
def check_child_feasibility(program: MPQCQP_Program, set_list: List[List[int]], combination_checker: CombinationTester) -> List[List[int]]:
    """
    Checks the feasibility of a list of active set combinations, if infeasible add to the combination checker and returns all feasible active set combinations

    :param program: An MPQP Program
    :param set_list: The list of active sets
    :param combination_checker: The combination checker that prunes
    :return: The list of all feasible active sets
    """
    output = list()
    for child in set_list:
        if program.check_feasibility(child):
            output.append(child)
        else:
            combination_checker.add_combo(child)

    return output

save = []
start_time = time.time()
for i in range(max_depth):
# if there are no other active sets to check break out of loop
# print(len(to_check))

    future_sets = list()
    # creates the list of feasible active sets
    feasible_sets = check_child_feasibility(program, to_check, murder_list)
    for child_set in feasible_sets:

        # soln = check_optimality(program, equality_indices=child_set)
        # The active set is optimal try to build a critical region

        # print(f"Child set: {child_set}")
        # if soln is not None:
        print('--'*50)
        sol =  program.check_optimality(child_set) # 检查最优性
        if sol is not None:
            critical_region = 1
            save.append(sol)
            print(child_set)
            # critical_region = gen_cr_from_active_set(program, child_set)  #TODO CR 是否是由 inactive set 组成?
            # Check the dimensions of the critical region
            if critical_region is not None:
                solution.add_region(critical_region)

        # propagate sets

        if i + 1 != max_depth:
            future_sets.extend(generate_children_sets(child_set, program.num_constraints(), murder_list))

    to_check = future_sets
end_time = time.time()
print("耗时: {:.2f}秒".format(end_time - start_time))

print('--'*60)

for element in save:
    print('**'*60)
    for key, values in element.items():
    # if isinstance(values, List):

        print(f"The {key}: {values}")
print(f"Number of Save is {len(save)}")

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) Gold 6140 CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 36 physical cores, 72 logical processors, using up to 32 threads

Optimize a model with 32 rows, 33 columns and 99 nonzeros
Model fingerprint: 0x3c91ee31
Model has 14 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e-16, 1e+03]
  RHS range        [1e-16, 1e+03]
  QRHS range       [9e-01, 8e+03]
Presolve removed 13 rows and 5 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 22 rows and 16 columns
Presolve time: 0.00s
Presolved: 56 rows, 29 columns, 169 nonzeros
Presolved model has 3 quadratic constraint(s)
Presolved model has 12 bilinear constraint(s)
Variable types: 29 continuous, 0 integer (0 binary)

Explo

In [ ]:
# for element in save:
#     print('**'*60)
#     for key, values in element.items():
#     # if isinstance(values, List):
#
#         print(f"The {key}: {values}")

In [ ]:
# feasible_sets = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 31]
# # feasible_sets.append(43)
# # for child_set in feasible_sets:
#
#     # soln = check_optimality(program, equality_indices=child_set)
#     # The active set is optimal try to build a critical region
#
#     # print(f"Child set: {child_set}")
#     # if soln is not None:
# print('--'*50)
# sol =  program.check_optimality(feasible_sets)
# if sol is not None:
#     print(f"Sol is; {sol}")
#     # critical_region = 1
#     #
#     # # critical_region = gen_cr_from_active_set(program, child_set)  #
#     # # Check the dimensions of the critical region
#     # if critical_region is not None:
#     #     solution.add_region(critical_region)
#
#     # propagate sets
#
#     # if i + 1 != max_depth:
#     #     future_sets.extend(generate_children_sets(child_set, program.num_constraints(), murder_list))

In [ ]:
program.num_equality_constraints()
